# Counting comparison

## Import libraries

In [13]:
import pandas as pd

## Configuration

In [14]:
CONFIG = {
    'date': '2023-03-28',
    'from_time': '07:00:00',
    'filter_class': ["car", "bus", "motorcycle", "truck", "truck_with_trailer", "truck_with_semitrailer", "delivery_van", "private_van"],
    'interval_length_min': 15 # In minutes
}

## Import Excel table

In [15]:
excel_table = pd.DataFrame(data=pd.read_excel('D:\SH-B75\ZaehlungAuswertung_B75_0700.xlsm', sheet_name='Zaehler', skiprows=range(1), usecols=['Klasse','Strom','Zeitstempel']))

## IDs

In [16]:
id_dict = {
    'id_to_class':{100:'pedestrian', 200:'bike', 300:'motorbike', 400:'car', 500:'car_with_trailer', 600:'delivery_van', 700:'truck', 800:'truck_with_trailer', 900:'truck_with_semitrailer', 1000:'bus', 1100:'others'},
    'id_to_from_section':{10: 'SOUTH', 20: 'SOUTH', 30: 'SOUTH', 40: 'EAST', 50: 'EAST', 60: 'EAST', 70: 'NORTH', 80: 'NORTH', 90: 'NORTH', 100: 'WEST', 110: 'WEST', 120: 'WEST', 130: 'SIDEWALK'},
    'id_to_to_section':{10: 'WEST', 20: 'NORTH', 30: 'EAST', 40: 'SOUTH', 50: 'WEST', 60: 'NORTH', 70: 'EAST', 80: 'SOUTH', 90: 'WEST', 100: 'NORTH', 110: 'EAST', 120: 'SOUTH', 130: 'SIDEWALK'}
}

## Formatting

In [18]:
# Renaming columns to match column names in flow_table
excel_table.rename(columns={'Klasse':'road_user_type', 'Zeitstempel':'time_interval'}, inplace=True)

# Time formatting
excel_table['time_interval'] = pd.Timestamp(CONFIG['date'] + 'T' + CONFIG['from_time']) + pd.to_timedelta(excel_table['time_interval'], unit='s')

# Replacing vehicle class IDs by the proper vehicle names
excel_table['road_user_type'] = excel_table['road_user_type'].map(id_to_class)

# Getting sections of origin and destination from "Strom" in two different columns
excel_table['from_section'] = excel_table['Strom'].map(id_to_from_section)
excel_table['to_section'] = excel_table['Strom'].map(id_to_to_section)

# Group by sections, time interval, and road user type
excel_table = excel_table.groupby(['from_section','to_section',pd.Grouper(freq=str(CONFIG['interval_length_min'])+'min', key='time_interval'),'road_user_type']).count().reset_index()

## Comparison results

### Comparison variables ideas:

- Difference of counting (brut / relative)
    - Overall
    - For all class/flow combination
    - For each vehicule class
- Graphs of counted number of vehicules per time intervall
    - Both graphs
    - Observed differences

In [7]:
from OTAnalytics.plugin_parser.excel_count_parser import ExcelCountParser

CONFIG = {
    'date': '2023-03-28',
    'from_time': '07:00:00',
    'filter_class': ["car", "bus", "motorcycle", "truck", "truck_with_trailer", "truck_with_semitrailer", "delivery_van", "private_van"],
    'interval_length_min': 15 # In minutes
}

id_dict = {
    'id_to_class':{100:'pedestrian', 200:'bike', 300:'motorbike', 400:'car', 500:'car_with_trailer', 600:'delivery_van', 700:'truck', 800:'truck_with_trailer', 900:'truck_with_semitrailer', 1000:'bus', 1100:'others'},
    'id_to_from_section':{10: 'SOUTH', 20: 'SOUTH', 30: 'SOUTH', 40: 'EAST', 50: 'EAST', 60: 'EAST', 70: 'NORTH', 80: 'NORTH', 90: 'NORTH', 100: 'WEST', 110: 'WEST', 120: 'WEST', 130: 'SIDEWALK'},
    'id_to_to_section':{10: 'WEST', 20: 'NORTH', 30: 'EAST', 40: 'SOUTH', 50: 'WEST', 60: 'NORTH', 70: 'EAST', 80: 'SOUTH', 90: 'WEST', 100: 'NORTH', 110: 'EAST', 120: 'SOUTH', 130: 'SIDEWALK'}
}

parser = ExcelCountParser()
excel_table = parser.excel_parser(path='D:\SH-B75\ZaehlungAuswertung_B75_0700.xlsm', id_dict=id_dict, CONFIG=CONFIG)